In [1]:
import math
import os
import random
import sys
import time
import numpy as np
import tensorflow as tf
import datetime
from tqdm import tqdm

In [2]:
# from text_data_2 import *
from text_data_no_bucket import *
from create_dict_2 import *
import model_seq2seq_2
from model_seq2seq_2 import Model

Loading dataset from ...
Loaded: 50004 words, 9999 training samples
Shuffling dataset...
[[946, 41, 9, 40, 363, 27356, 10, 13, 363, 12, 4, 1988, 28, 4817, 132, 234, 3230, 596, 808, 81, 126, 851, 0, 27, 14187, 3967, 423, 721, 144, 795, 9878, 18, 5, 479, 1851, 0, 7, 6, 139, 0, 93, 0, 117, 9, 24, 478, 0, 0, 30, 44, 123, 9, 12, 15, 9, 2003, 203, 4, 667, 239, 63, 8, 1035, 7], [29, 24, 778, 624, 8, 60, 47, 5, 6, 10121, 699, 14, 13, 4817, 184, 277, 5070, 5, 208, 2943, 851, 64, 0, 5, 766, 33, 5, 8, 57, 73, 37, 21, 332, 8, 74, 0, 192, 46, 198, 0, 19, 0, 22, 3105, 178, 17, 0, 0, 207, 68, 21, 52, 655, 13, 33, 13, 3684, 18, 6, 3934, 796, 71, 351, 43], [15, 987, 259, 607, 4, 7, 6, 1356, 27, 6, 5, 358, 5, 3, 34, 15, 13, 271, 1834, 22, 12, 13039, 0, 43596, 31, 77, 1341, 1141, 17, 118, 26, 774, 6, 15, 16, 0, 5, 444, 3, 0, 70, 0, 417, 195, 1417, 894, 0, 0, 6, 987, 192, 891, 8, 675, 26, 1348, 3, 21, 3826, 90, 455, 11, 6, 110], [239, 8, 74, 7, 126, 354, 4, 92, 143, 14, 265, 745, 206, 80, 11, 33, 91, 20, 

In [3]:
tf.app.flags.DEFINE_float('learning_rate',0.5,'Learning rate')
tf.app.flags.DEFINE_float('learning_rate_decay_factor',0.99,'Learning rate decays by this much')
tf.app.flags.DEFINE_float('max_gradient_norm',5.0,'Clip gradients to this norm')
tf.app.flags.DEFINE_integer('batch_size',64,'Batch size for training')
tf.app.flags.DEFINE_integer('size',20,'Size of each layer') # the number of unfolded LSTM units in each layer(maybe too larger)..? or the dimension of hidden vector?
# the number of unfolded LSTM units could have big influence on the time need to build the model, 512 units require a long time
tf.app.flags.DEFINE_integer('num_layers',3,'Number of layers')
tf.app.flags.DEFINE_integer('num_epochs',3,'maximum number of epochs to run')
tf.app.flags.DEFINE_integer('vocab_size',50000,'Vocabulary size, words with lower frequency are regarded as unknown')
tf.app.flags.DEFINE_string('train_samples_path','ubuntu_train_samples.pkl','Processed training samples')
tf.app.flags.DEFINE_string('valid_samples_path','ubuntu_valid_samples.pkl','Processed validation samples')
# tf.app.flags.DEFINE_string('test_samples_path','ubuntu_test_samples.pkl','Processed test samples')
rootDir = "/Users/hanzhichao/Documents/ETH_Courses/DeepLearning/project/"
tf.app.flags.DEFINE_string('train_data_path',rootDir+'data/ubuntu/train.csv','Training data')
tf.app.flags.DEFINE_string('valid_data_path',rootDir+'data/ubuntu/valid.csv','Validation data')
# tf.app.flags.DEFINE_string('test_data_path','data/ubuntu/test.csv','Test data')
tf.app.flags.DEFINE_string('sorted_list_path','ubuntu_freqlist.pkl','List of words sorted by frequency')
tf.app.flags.DEFINE_string('dialog_path',rootDir+'data/ubuntu/dialogs/','Directory of raw ubuntu dialogues')
tf.app.flags.DEFINE_integer('playDataset',20,'Display random samples from data')
tf.app.flags.DEFINE_integer('max_train_data_size',0,'Limit on the size of training data (0: no limit)')
tf.app.flags.DEFINE_integer('steps_per_checkpoint',500,'Number of training steps between checkpoints')
tf.app.flags.DEFINE_boolean('decode',False,'Set to True for interactive decoding')

In [4]:
FLAGS = tf.app.flags.FLAGS

In [5]:
_buckets = [(10,20),(20,40),(30,60),(40,80),(50,100)]

In [6]:
def create_model(session):
    model = Model(FLAGS)
    # TODO: save and restore model to/from hard disk
    Load_model = False
    if Load_model:
        model.saver.restore(session, FILA_NAME)
    else:
        session.run(tf.initialize_all_variables())
    return model

In [7]:
reload(model_seq2seq_2)
from model_seq2seq_2 import Model

In [8]:
sess = tf.Session()

In [9]:
model = Model(FLAGS)

model creation...
3
starting building network...
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
network built...


In [10]:
model.encoderInputs

[<tf.Tensor 'placeholder_encoder/Placeholder:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_1:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_2:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_3:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_4:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_5:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_6:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_7:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_8:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_9:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_10:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_11:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_12:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder

In [11]:
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [12]:
FLAGS.train_samples_path

'ubuntu_train_samples.pkl'

In [13]:
FLAGS.dialog_path

'/Users/hanzhichao/Documents/ETH_Courses/DeepLearning/project/data/ubuntu/dialogs/'

In [14]:
FLAGS.sorted_list_path

'ubuntu_freqlist.pkl'

In [15]:
train_set = TextData(FLAGS.train_samples_path, FLAGS.train_data_path, FLAGS.sorted_list_path, FLAGS.dialog_path, FLAGS.vocab_size, FLAGS.playDataset)

Loading dataset from ...
Loaded: 50004 words, 9999 training samples
Randomly play samples:
Context: locking your front door with scotch tape ... . its only for testing the wifi disconnects and by the way , wpa2 is also unsafe
Utterance: alrighty : ) just sayin '
Context: hi there what means `` ../ '' and `` ./ '' ? ? parent directory and current directory
Utterance: and boohbah thank you so much
Context: vmware issue ? how would it ? you are n't emulating a wireless connection can it be set ?
Utterance: i do n't use usb wireless or vmware i think you need to use bridged networking
Context: i do not understand your reasoning ... if grub is hosed , why would you not be able to ssh to it from live ?
Utterance: oh from a live cd ... sorry . no it wont , you should always use purge on
Context: i have not really followed it as a result . was curious if anyone here knew if there are updates with <unknown> 3 directly .
Utterance: does it work in sound recorder
Context: heyho ! i 'm running ubu

In [16]:
dev_set = TextData(FLAGS.valid_samples_path, FLAGS.valid_data_path, FLAGS.sorted_list_path, FLAGS.dialog_path, FLAGS.vocab_size, FLAGS.playDataset)

Loading dataset from ...
Loaded: 50004 words, 9999 training samples
Randomly play samples:
Context: can change your mac address with macchanger - correct but it will change it on the physical nic this may help , http : <unknown>
Utterance: - unfortunately that does n't let me set the mac address
Context: work but then i get bootmgr is missing , and i have a sinking feeling i will need a windows 7 disc to repair that
Utterance: yes , that might explain why it has n't been picked up
Context: problem i am having http : //ubuntuforums.org/showthread.php ? <unknown> look into 'dmesg ' ; if there 's nothing suspicious , then it 's your mice
Utterance: and i do n't have this problem in windows 7 i should note that my mouse still works
Context: probably fine for what you want to do . i really appreciate the assistance is your image mounted ? what filesystem type is it ?
Utterance: i am not sure what kind of filesystem it is ... i do n't believe its mounted ,
Context: manager , and i have selec

In [17]:
step_time, loss = 0.0, 0.0
current_step = 0
previous_loss = []

In [18]:
FLAGS.num_epochs = 3
FLAGS.vocab_size = 50004

In [ ]:
for e in range(FLAGS.num_epochs):
    print ("\n")
    print("----- Epoch {}/{} ; (lr={}) -----".format(e+1, FLAGS.num_epochs, model.learning_rate))
    batches = train_set.get_batches(FLAGS.batch_size)
    print( len(batches))
    for nextBatch in tqdm(batches, desc="Training"):
        ops, feedDict = model.step(nextBatch) # for nextBatch:  decoderSeqs and targetSeqs are very similar, see DeepQA/textdata.py/Line 121-122
        TRAINING = 1
        _, step_loss = sess.run(ops, feedDict)




----- Epoch 1/3 ; (lr=0.5) -----
Shuffling dataset...


Training:   0%|          | 0/157 [00:00<?, ?it/s]

157


Training:  13%|█▎        | 20/157 [01:06<07:27,  3.26s/it]

In [20]:
nextBatch.encoderSeqs

[[301, 5, 1435, 35, 20, 0, 12, 0, 11, 7, 6, 347, 23, 33, 312],
 [5, 1818, 12, 18, 1070, 0, 2319, 0, 28, 442, 4, 5675, 83, 2404, 18],
 [924, 24, 1079, 4, 339, 203, 38227, 0, 1197, 15, 19, 425, 15, 36, 3],
 [6, 13996, 2092, 18, 11, 7, 34, 0, 11, 932, 21, 7, 9, 7, 40],
 [11, 14, 22, 5, 47, 61, 15, 0, 19, 2711, 58, 5, 33, 83, 3],
 [75, 76, 1233, 13142, 191, 4, 1205, 0, 89, 31, 70, 144, 631, 34, 10],
 [36, 9, 7, 8, 10, 19, 10, 0, 6, 32, 24, 114, 10, 39, 9],
 [38, 445, 17640, 50, 13, 12, 120, 0, 11, 808, 4940, 25, 28, 13, 13],
 [244, 5256, 4143, 135, 39, 169, 49, 17, 72, 39, 363, 9, 13, 178, 1023],
 [7, 7, 6, 168, 12, 10533, 19390, 11129, 12, 13, 264, 13, 86, 4846, 146],
 [171, 5, 34, 73, 111, 32, 295, 6, 9578, 46, 83, 26, 336, 3, 15],
 [74, 3335, 11, 5, 4, 1795, 50, 23, 167, 285, 35, 275, 528, 6, 4],
 [5, 745, 181, 145, 23, 17, 18, 11, 8, 430, 3419, 52, 8, 5, 19],
 [546, 10, 8091, 9, 442, 12, 11, 170, 16, 52, 381, 5, 10689, 516, 7],
 [3, 91, 7, 68, 20, 234, 156, 221, 246, 3325, 20, 32, 42, 

In [22]:
print len(nextBatch.weights)
print len(nextBatch.weights[0])

20
15
